# BoxOfficeMojo Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


#from boxofficemojo import boxofficemojo

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-12-25 12:03:55.768581


# Get/Parse/Merge/Process Box Office Mojo Data

In [6]:
bom = boxofficemojo()
_, _ = clock("Last Run")

Current Time is Tue Dec 25, 2018 12:04:25 for Last Run


In [7]:
bom.getBoxOfficeMojoWeekendData(1982, 1983, debug=True)
_, _ = clock("Last Run")

Data Directory: /Users/tgadfort/Documents/code/movies/boxoffice.com/data
Current Time is Tue Dec 25, 2018 12:04:25 for Last Run


In [8]:
bom.parseBoxOfficeMojoResults(debug=False)
_, _ = clock("Last Run")

Saving 52 weekends of movie data to /Users/tgadfort/Documents/code/movies/boxoffice.com/results/1982.json
Saving 52 weekends of movie data to /Users/tgadfort/Documents/code/movies/boxoffice.com/results/1983.json
Saving 0 weekends of movie data to /Users/tgadfort/Documents/code/movies/boxoffice.com/results/1984.json
Saving 0 weekends of movie data to /Users/tgadfort/Documents/code/movies/boxoffice.com/results/1985.json
Saving 0 weekends of movie data to /Users/tgadfort/Documents/code/movies/boxoffice.com/results/1986.json
Saving 0 weekends of movie data to /Users/tgadfort/Documents/code/movies/boxoffice.com/results/1987.json
Saving 0 weekends of movie data to /Users/tgadfort/Documents/code/movies/boxoffice.com/results/1988.json
Saving 0 weekends of movie data to /Users/tgadfort/Documents/code/movies/boxoffice.com/results/1989.json
Saving 0 weekends of movie data to /Users/tgadfort/Documents/code/movies/boxoffice.com/results/1990.json
Saving 0 weekends of movie data to /Users/tgadfort/Do

In [9]:
bom.mergeBoxOfficeMojoResults(debug=True)
_, _ = clock("Last Run")

Found 36 files in the results directory
Saving 36 years of movie data to /Users/tgadfort/Documents/code/movies/boxoffice.com/results.json
Current Time is Tue Dec 25, 2018 12:04:30 for Last Run


In [10]:
bom.processBoxOfficeMojo(debug=True)
_, _ = clock("Last Run")

----> 1989  (Top 25 Movies) <----


----> 1985  (Top 25 Movies) <----


----> 1993  (Top 25 Movies) <----


----> 2014  (Top 25 Movies) <----


----> 2002  (Top 25 Movies) <----


----> 2003  (Top 25 Movies) <----


----> 2015  (Top 25 Movies) <----


----> 1992  (Top 25 Movies) <----


----> 1984  (Top 25 Movies) <----


----> 1988  (Top 25 Movies) <----


----> 1983  (Top 25 Movies) <----
('E.T.: The Extra-Terrestrial', 352905920.0)
('Return of the Jedi', 249253614.0)
('Tootsie', 176188774.0)
('An Officer and a Gentleman', 118258671.0)
('Flashdance', 90463574.0)
('Trading Places', 88621914.0)
('48 HRS.', 72416908.0)
('WarGames', 71654691.0)
('Staying Alive', 62021021.0)
('Mr. Mom', 61339026.0)
('Risky Business', 61178400.0)
("National Lampoon's Vacation", 60684903.0)
('Octopussy', 59321214.0)
('Superman III', 58435322.0)
('The Verdict', 51771248.0)
('Gandhi', 51669867.0)
('Never Say Never Again', 51284244.0)
('The Toy', 46182747.0)
('Blue Thunder', 42313354.0)
('Sudden Impact', 41852

In [5]:
import re
from time import sleep
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from fsUtils import setDir, isDir, mkDir, setFile, isFile, setSubFile
from fileUtils import getBaseFilename
from searchUtils import findSubPatternExt, findPatternExt, findExt
from strUtils import convertCurrency
from webUtils import getWebData, getHTML
from movieDB import movieDB
from os import getcwd
import operator


##############################################################################################################################
# Box Office Mojo
##############################################################################################################################
class boxofficemojo(movieDB):
    def __init__(self, basedir=None):
        self.name = "boxoffice.com"
        movieDB.__init__(self, dbdir=self.name)
    
    
    
    ###########################################################################################################################
    # Get Box Office Weekend Files
    ###########################################################################################################################
    def downloadBoxOfficeMojoWeekendData(self, year, week, outdir, debug=False):
        yname = str(year)
        if week < 10:
            wname = "0"+str(week)
        else:
            wname = str(week)

        url="http://www.boxofficemojo.com/weekend/chart/?yr="+yname+"&wknd="+wname+"&p=.htm"
        savename = setFile(outdir, yname+"-"+wname+".p")
        if isFile(savename): return
        if debug:
            print("Downloading/Saving {0}".format(savename))
        getWebData(base=url, savename=savename, useSafari=False)
        sleep(2)

    def getBoxOfficeMojoWeekendData(self, startYear = 1982, endYear = 1982, debug=False):
        outdir = self.getDataDir()
        if debug:
            print("Data Directory: {0}".format(outdir))
        #outdir = setDir(getBoxOfficeDir(), "data")
        if not isDir(outdir): mkDir(outdir)
        years  = range(int(startYear), int(endYear)+1)
        weeks  = range(1,53)
        for year in years:
            for week in weeks:
                self.downloadBoxOfficeMojoWeekendData(year, week, outdir, debug)
                
                
    
    
    ###########################################################################################################################
    # Parse Box Office Weekend Files
    ###########################################################################################################################
    def parseBoxOfficeMojo(self, ifile, debug=False):
        htmldata = getFile(ifile)
        bsdata   = getHTML(htmldata)
        tbl = None
        for table in bsdata.findAll("table"):
            if tbl:
                break
            for tr in table.findAll("tr"):
                if len(tr) >= 10:
                    tbl = table
                    break
                else:
                    break

        #print len(tbl)
        keys = []
        data = []
        for i,tr in enumerate(tbl):
            vals = []
            if i == 0:
                for j,td in enumerate(tr.findAll("td")):
                    for ref in td.findAll("a"):
                        key = ref.string
                        keys.append(key)
            else:
                if len(tr) <= 1: continue
                #print "\n\n\nNext...."
                #print tr
                #print "  tr-->",tr,'\t',len(tr)
                #print i,tr,len(data)
                for j,td in enumerate(tr.findAll("td")):
                    if td.string == None:
                        continue
                    try:
                        if re.search("TOTAL \((\d+) MOVIES\)", td.string):
                            break
                    except:
                        print(j,td.string)
                        raise()
                    key = keys[j]
                    val = td.string
                    vals.append(val)
                    #print j,'\t',keys[j],'\t',td.string
                if len(vals) == 0: break
                if len(vals) != len(keys):
                    print("Mismatch with keys/data")
                    print(len(keys),'\t',keys)
                    print(len(vals),'\t',vals)
                    break
                else:
                    data.append(vals)

        if debug:
            print("Found",len(data),"movies from",ifile)
        return data


    def parseBoxOfficeMojoResults(self, startYear = 1982, endYear = 2017, debug=False):
        outdir = self.getDataDir()
        resultsdir = self.getResultsDir()
        
        if endYear == None: endYear = startYear
        years    = range(int(startYear), int(endYear)+1)
        for year in years:
            retval = []
            files  = findPatternExt(outdir, pattern=str(year), ext=".p")
            for ifile in files:
                result = self.parseBoxOfficeMojo(ifile, debug=debug)
                retval.append(result)

            savename = setFile(resultsdir, str(year)+".json")
            print("Saving",len(retval),"weekends of movie data to",savename)
            saveFile(savename, retval)
                
                
    
    
    ###########################################################################################################################
    # Merge Box Office Weekend Files
    ###########################################################################################################################
    def mergeBoxOfficeMojoResults(self, debug=False):
        retval = {}
        files  = findExt(self.getResultsDir(), ext=".json")
        if debug:
            print("Found {0} files in the results directory".format(len(files)))
        for ifile in files:
            year = getBaseFilename(ifile)
            data = getFile(ifile)
            retval[year] = data

        savename = setFile(self.getMovieDBDir(), "results.json")
        if debug:
            print("Saving",len(retval),"years of movie data to",savename)
        saveFile(savename, retval)
                
                
    
    
    ###########################################################################################################################
    # Merge Box Office Weekend Files
    ###########################################################################################################################
    def processBoxOfficeMojo(self, debug=False):
        outdir   = self.getMovieDBDir()
        savename = setFile(outdir, "results.json")

        data = getFile(savename)
        movies = {}
        yearlyData = {}
        for i,year in enumerate(data.keys()):
            movies[year] = {}
            ydata = data[year]
            
            for wdata in ydata:
                for mdata in wdata:
                    movie  = mdata[2]
                    retval = re.search("\((\d+)\)",movie)
                    if retval:
                        stryear  = retval.group()
                        movie = movie.replace(stryear, "").strip()

                    gross  = convertCurrency(mdata[9])
                    weekly = convertCurrency(mdata[4])
                    money  = max(gross, weekly)
                    if movies[year].get(movie) == None:
                        movies[year][movie] = money
                    else:                    
                        movies[year][movie] = max(money, movies[year][movie])

            yearlyData[year] = sorted(movies[year].items(), key=operator.itemgetter(1), reverse=True)
            print("---->",year," (Top 25 Movies) <----")
            for item in yearlyData[year][:25]:
                print(item)
            print('\n')

        savename = setFile(outdir, "{0}.json".format(self.name))
        print("Saving",len(yearlyData),"yearly results to",savename)
        saveFile(savename, yearlyData)